# Rossmann Sales Prediction

## About Sales Prediction

In the world of retail, accurate sales prediction is key for companies like Rossmann. Sales prediction is the practice of leveraging historical data and data-driven methodologies to anticipate future sales trends, enabling businesses to make informed decisions on inventory management, marketing strategies, and resource allocation. 

## About Rossmann

Rossmann, a prominent European drug store and retail chain, plays a vital role in the daily lives of countless customers across multiple countries. With a vast array of products, from cosmetics and personal care items to household essentials, Rossmann has become a go-to destination for shoppers seeking quality goods at competitive prices. 

In this data science project, I delve into the challenge of forecasting Rossmann store revenues, a task with significant implications for the company's operations and decision-making. By leveraging data-driven insights and predictive models, I aim to uncover patterns and trends that can help Rossmann optimize its business strategies and improve its performance.

## Business Problem

**Business Challenge: Developing a Sales Prediction Model**

A sales prediction model for the following six weeks will be developed in order to predict how much money each store of the company will have available to spend on their renovation.

## 0. Imports

### 0.1. Libraries

In [1]:
### Data Manipulation
import pandas                   as pd
import numpy                    as np

### Other Libraries
import inflection
import datetime 
import warnings

### 0.2. Settings

In [2]:
### Ignoring warnings
warnings.filterwarnings('ignore')

### Pandas Settings
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### 0.3. Functions

In [3]:
### Data manipulation functions

def df_shape(df):
    """
    Summary: The function prints the number of rows and columns of a given DataFrame.

    Args: DataFrame we want to know more about.
    
    Returns: number of rows and columns of the DataFrame.
    """

    print('Number of rows: {:,}'.format(df.shape[0]))
    print('Number of columns: {:,}'.format(df.shape[1]))

    return None

### Pre-processing
def df_na_values(dataframe):
    """
    Summary: This function displays the name of the columns that have NA values (total and percentage) of a dataframe

    Args: Dataframe 

    Returns: Dataframe with name of feature, NA values, NA% values
    """
    # Calculating
    na_values = pd.DataFrame(
        {'NA':  dataframe.isna().sum(), 
         'NA%': (dataframe.isna().sum()) / (dataframe.shape[0]) * 100}
    ).sort_values('NA', ascending=False)

    # Filtering values
    na_values = na_values[na_values['NA'] != 0]

    # Formatting
    na_values['NA'] = na_values['NA'].apply(lambda x: f'{x:,.0f}')
    na_values['NA%'] = na_values['NA%'].apply(lambda x: f'{x:.2f}%')

    return na_values

def columns_to_snake_case(dataframe):
    """
    Summary: This function transforms the column names to snake_case style.

    Args: Dataframe with incorrect column names.

    Returns: None.
    """
    # List of columns
    old_columns = dataframe.columns.tolist()
    # Lambda function
    snake_case = lambda x: inflection.underscore(x)
    # Assigning new column names to DataFrame
    dataframe.columns = list(map(snake_case, old_columns))

    return None

def comp_nan_values(df, month_column, year_column):
    """
    Summary: This function fill out the NaN values on monthly and yearly competition features 
    Args: Main Dataframe, Monthly column, Yearly column
    Returns: None
    """
    # Monthly
    df[month_column] = df.apply(lambda x: x['date'].month if np.isnan(x[month_column]) 
    else x[month_column], axis=1)
    
    # Yearly
    df[year_column] = df.apply(lambda x: x['date'].year if np.isnan(x[year_column])
    else x[year_column], axis=1)

    return None

def promo2_nan_values(df, week_column, year_column):
    """
    Summary: This function fill out the NaN values on weekly and yearly promo2 features 
    Args: Main Dataframe, Weekly column, Yearly column
    Returns: None
    """
    # Monthly
    df[week_column] = df.apply(lambda x: x['date'].week if np.isnan(x[week_column]) 
    else x[week_column], axis=1)
    
    # Yearly
    df[year_column] = df.apply(lambda x: x['date'].year if np.isnan(x[year_column])
    else x[year_column], axis=1)

    return None

def to_int_64(dataframe, list_of_columns):
        """
        Summary: This function transform the data type of columns (list of columns) to int64
        Args: Main Dataframe, List of columns to be modified
        Returns: None
        """
        for column in list_of_columns:
                dataframe[column] = dataframe[column].astype('int64')

        return None

### 0.4. Dataset

In [4]:
# Importing data
df_store = pd.read_csv('../data/raw/store.csv')
df_sales = pd.read_csv('../data/raw/train.csv')

# Merging dataframes
df_merge = pd.merge(df_sales, df_store, how='left', on='Store')

# # Export merge dataframe
df_merge.to_csv('../data/interim/merged_dataset.csv', index=False)

# Working with df as main dataframe
df = pd.read_csv('../data/interim/merged_dataset.csv')

## 1. Data Description

In [5]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.00,11.00,2007.00,1,13.00,2010.00,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.00,12.00,2006.00,1,14.00,2011.00,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.00,9.00,2009.00,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.00,4.00,2015.00,0,NaN,NaN,NaN


### 1.1. Data dimensions

In [6]:
df_shape(df)

Number of rows: 1,017,209
Number of columns: 18


### 1.2. Columns

In [7]:
df.columns.to_list()

['Store',
 'DayOfWeek',
 'Date',
 'Sales',
 'Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval']

### 1.3. Data types

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

### 1.4. Checking NaN

In [9]:
df_na_values(df)

,NA,NA%
PromoInterval,"508,031",49.94%
Promo2SinceYear,"508,031",49.94%
Promo2SinceWeek,"508,031",49.94%
CompetitionOpenSinceYear,"323,348",31.79%
CompetitionOpenSinceMonth,"323,348",31.79%
CompetitionDistance,"2,642",0.26%


## 2. Data Exploration and Preprocessing

### 2.1. Renaming columns

In [10]:
# Renaming columns to snake_case style
columns_to_snake_case(df)

# Shortening columns names
mapping = {
    'competition_distance':         'comp_distance',
    'competition_open_since_month': 'comp_open_month',
    'competition_open_since_year':  'comp_open_year'}

df.rename(columns=mapping, inplace=True)

# Visualizing column names
df.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'comp_distance', 'comp_open_month', 'comp_open_year', 'promo2',
       'promo2_since_week', 'promo2_since_year', 'promo_interval'],
      dtype='object')

### 2.2. Datetime variable

In [11]:
# Column 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

### 2.3. Filling out NaN values on date-related columns

**Competition Open Since Month & Year**

The missing values in `competition_open_since_month` and `competition_open_since_year` will be replaced with the month and year from `date` column

In [12]:
comp_nan_values(df, 'comp_open_month', 'comp_open_year')

**Promo2 Since Week & Year**

The missing values in `promo2_since_week` and `promo2_since_year` are replaced the same way as above, with data from the `date` column.

In [13]:
promo2_nan_values(df, 'promo2_since_week', 'promo2_since_year')

**Promo Interval**

The missing values in `promo_interval` will be replaced as zeros, considering that the promotion never took place.

In [14]:
df['promo_interval'].fillna(0, inplace=True)

In [15]:
# Checking missing values
df_na_values(df)

,NA,NA%
comp_distance,"2,642",0.26%


**Competition Distance**

The missing values in `competition_distance` problably indicates that there are no near competitors or maybe they are too far away. 

Therefore, a calculus will be used to fill the missing values after the train-test split.

### 2.4. Changing data types

In [16]:
# Transforming float to int64
list_to_int_64 = ['comp_open_month', 'comp_open_year', 'promo2_since_week', 'promo2_since_year']

to_int_64(df, list_to_int_64)

### 2.5. Reasonable Changes

In [17]:
# Since Rossmann was founded in March 17 1972, if "competition_since_dates" is less than founding date, it will be replaced by it
# 349 occurrances of 'comp_open_year' < 1972
df.loc[df['comp_open_year'] < 1972, 'comp_open_year'] = 1972

### 2.6. Features Creation for EDA

#### 2.6.1. Date and time features

In [18]:
df['year']       = df['date'].dt.year # Year
df['month']      = df['date'].dt.month # Month
df['week']       = df['date'].dt.weekofyear # Week
df['day']        = df['date'].dt.day # Day
df['year_week']  = df['date'].dt.strftime('%Y-%W') # Year-Week

#### 2.6.2. Competition features

In [19]:
# New feature based on year and month 
df['comp_since'] = df.apply(
    lambda x: datetime.datetime(
        year  =x['comp_open_year'], 
        month =x['comp_open_month'], day=1), axis=1)

# Time of competition in days
df['comp_time_delta'] = df['date'] - df['comp_since']

# Corrections where 'comp_since' is higher than 'date'
df.loc[df['comp_since'] > df['date'], 'comp_time_delta'] = datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)

# Time of competition in months
df['comp_time_month'] = df['comp_time_delta'].astype('timedelta64[M]')

# Dropping unnecessary columns
df.drop(['comp_open_month', 'comp_open_year'], axis=1, inplace=True)

#### 2.6.3. Promotion features

In [20]:
# Time of promotion
df['promo_since'] = df['promo2_since_year'].astype(str) + '-' + df['promo2_since_week'].astype(str)

# Promo as datetime
df['promo_since'] = df['promo_since'].apply(
    lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))

# Time of promo in days
df['promo_time_delta'] = df['date'] - df['promo_since']

# Corrections
df.loc[df['promo_since'] > df['date'], 'promo_time_delta'] = datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)

# Time of promotion in months
df['promo_time_month'] = df['promo_time_delta'].astype('timedelta64[M]')

# Dropping unnecessary columns
df.drop(['promo2_since_week', 'promo2_since_year', 'promo_interval'], axis=1, inplace=True)

#### 2.6.4. Categorical Variables

In [21]:
# Assortment
df['assortment'] = df['assortment'].apply(
    lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

# State Holiday
df['state_holiday'] = df['state_holiday'].apply(
    lambda x: 'public_holiday' if x == 'a' 
    else 'easter_holiday' if x == 'b' 
    else 'christmas' if x == 'c' 
    else 'regular_day' )

## Exporting Processed Dataset

In [22]:
df.to_parquet('../data/processed/preprocessed_dataset.parquet', index=False)

In [23]:
df

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,comp_distance,promo2,year,month,week,day,year_week,comp_since,comp_time_delta,comp_time_month,promo_since,promo_time_delta,promo_time_month
0,1,5,2015-07-31,5263,555,1,1,regular_day,1,c,basic,1270.00,0,2015,7,31,31,2015-30,2008-09-01,2524 days,82.00,2015-07-27,4 days,0.00
1,2,5,2015-07-31,6064,625,1,1,regular_day,1,a,basic,570.00,1,2015,7,31,31,2015-30,2007-11-01,2829 days,92.00,2010-03-22,1957 days,64.00
2,3,5,2015-07-31,8314,821,1,1,regular_day,1,a,basic,14130.00,1,2015,7,31,31,2015-30,2006-12-01,3164 days,103.00,2011-03-28,1586 days,52.00
3,4,5,2015-07-31,13995,1498,1,1,regular_day,1,c,extended,620.00,0,2015,7,31,31,2015-30,2009-09-01,2159 days,70.00,2015-07-27,4 days,0.00
4,5,5,2015-07-31,4822,559,1,1,regular_day,1,a,basic,29910.00,0,2015,7,31,31,2015-30,2015-04-01,121 days,3.00,2015-07-27,4 days,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,public_holiday,1,a,basic,1900.00,1,2013,1,1,1,2013-00,2014-06-01,0 days,0.00,2013-07-29,0 days,0.00
1017205,1112,2,2013-01-01,0,0,0,0,public_holiday,1,c,extended,1880.00,0,2013,1,1,1,2013-00,2006-04-01,2467 days,81.00,2012-12-31,1 days,0.00
1017206,1113,2,2013-01-01,0,0,0,0,public_holiday,1,a,extended,9260.00,0,2013,1,1,1,2013-00,2013-01-01,0 days,0.00,2012-12-31,1 days,0.00
1017207,1114,2,2013-01-01,0,0,0,0,public_holiday,1,a,extended,870.00,0,2013,1,1,1,2013-00,2013-01-01,0 days,0.00,2012-12-31,1 days,0.00


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype          
---  ------            --------------    -----          
 0   store             1017209 non-null  int64          
 1   day_of_week       1017209 non-null  int64          
 2   date              1017209 non-null  datetime64[ns] 
 3   sales             1017209 non-null  int64          
 4   customers         1017209 non-null  int64          
 5   open              1017209 non-null  int64          
 6   promo             1017209 non-null  int64          
 7   state_holiday     1017209 non-null  object         
 8   school_holiday    1017209 non-null  int64          
 9   store_type        1017209 non-null  object         
 10  assortment        1017209 non-null  object         
 11  comp_distance     1014567 non-null  float64        
 12  promo2            1017209 non-null  int64          
 13  year              1017209 n